# Testing the Apache Spark cluster with pyspark

pyspark is an lib designed to ease the communication between python an Apache Spark cluster. This notebook is running inside a jupyter/pyspark-notebook pod with some configurations and enviromnents variables, so it can communicate with the cluster master through the kubernetes.

## 1 Configure pyspark

Run the cells bellow to configure a session inside the Apache Spark cluster

### 1.1 make the necessary importations

In [1]:
import pyspark
import socket
import random

### 1.2 create the session
use the socket lib to get the ip adress of this pod inside the kubernetes cluster

In [2]:
hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
print(ip_address)

10.42.0.20


then use this ip address and the service created to expose the master pod to configure a new session in the cluster

In [3]:
conf = pyspark.SparkConf()
conf.setAppName("simple task")
conf.set("spark.driver.host",ip_address)
conf.setMaster("spark://spark-master-service:7077")
sc = pyspark.SparkContext(conf=conf)

## 2 Run some test

### 2.1 first run
try to run the cell bellow to send a pi estimatin task to the cluster

In [4]:
%%time
def inside(p):
    x, y = random.random(), random.random()
    return 1 if x*x + y*y < 1 else 0
NUM_SAMPLES = 5000000
count = sc.parallelize(range(0, NUM_SAMPLES)).map(inside).reduce(lambda a, b: a + b)
print("[PYTHON SYSTEM OUT]Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

[PYTHON SYSTEM OUT]Pi is roughly 3.141670
CPU times: user 14.4 ms, sys: 7.43 ms, total: 21.8 ms
Wall time: 8.66 s


### 2.2 second run
come back to the first notebook and scale the cluster to 2 workers, then, run the code bellow using 2 workers and see the difference beetwen the time that the cluster take to run the code.

In [21]:
%%time
def inside(p):
    x, y = random.random(), random.random()
    return 1 if x*x + y*y < 1 else 0
NUM_SAMPLES = 5000000
count = sc.parallelize(range(0, NUM_SAMPLES)).map(inside).reduce(lambda a, b: a + b)
print("[PYTHON SYSTEM OUT]Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

[PYTHON SYSTEM OUT]Pi is roughly 3.141234
CPU times: user 10.1 ms, sys: 8.7 ms, total: 18.8 ms
Wall time: 4.44 s


## 3 Delete the cluster session
Stop the spark session and get back to the other notebook to delete the cluster

In [5]:
sc.stop()